In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

## Prearacion de los datos

Aquí construiremos vectores de 1 y 0 marcando a cada persona si ha ido de viaje a un pais o no

In [2]:
data = pd.read_table('marketing_pax_clean.csv',sep='^',header=0,decimal='.',index_col=0)

/home/pc/.local/lib/python2.7/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data.head()

,rloc,full_name,departure_date_leg,board_point,board_country_code,off_point,off_country_code
1,612666,Z/MZFFUFGDGZQ,2014-04-01,BRU,BE,LNZ,AT
3,827666,T/HUYMXMTGOTMUM,2013-02-02,BRU,BE,LIL,FR
4,8PD666,YU/YUDVMMTHQZEMX,2013-10-25,BRU,BE,MMX,SE
7,0QB666,VAZMFTMZBQSXQD/,2014-04-01,BRU,BE,KTM,NP
10,13J666,QZ/WMDUYUQFFUZ,2013-01-17,ALG,DZ,BRU,BE


In [24]:
#agrupamos todos los viajes que realiza una persona en un mismo registro

In [4]:
grouped = data.groupby('full_name')['off_country_code'].apply(lambda x: "%s" % '|'.join(x)).reset_index()

In [5]:
grouped.head()

,full_name,off_country_code
0,ADHAWWU YMDSMDQFMOTDUEFQZEQZ/FGGXM,BE|SE
1,AEEU MZFQDAXMTFU/VAGZU,DZ
2,AEOMD IUXTQXYWMDXEEAZ/MMWQ,SE|BE
3,AEOMD SGZZMDEEDQUZTMDP/NVADZ,DE|BE
4,AEOMD VATMZZQEYMDFUZEEAZ/XMDE,BE


In [25]:
#convertimos cada regristro de todos los viajes en un vector de 0,1 marcando si ha ido de viaje al pais o no
#para ello utilizamos la funcion get_dummies

In [6]:
datos = pd.concat([grouped,grouped.off_country_code.str.get_dummies(sep = '|')],axis = 1)

In [7]:
datos.head()

,full_name,off_country_code,AT,BE,BG,CA,CH,CN,CV,CY,...,PT,RU,SA,SE,TN,TR,UA,US,UZ,ZM
0,ADHAWWU YMDSMDQFMOTDUEFQZEQZ/FGGXM,BE|SE,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,AEEU MZFQDAXMTFU/VAGZU,DZ,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AEOMD IUXTQXYWMDXEEAZ/MMWQ,SE|BE,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,AEOMD SGZZMDEEDQUZTMDP/NVADZ,DE|BE,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AEOMD VATMZZQEYMDFUZEEAZ/XMDE,BE,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Creacion de raiting y recomendacion de viaje

Ahora lo que vamos hacer es coger y agrupar todos los vectores que sean de la misma forma y contar cuantas veces aparecen, ese conteo sera el peso o raiting del vector.
Una vez agrupados haremos un kneighbours para ver que vectores son los mas parecidos al que queremos predecir.
Finalmente con los vectores parecidos calcularemos el coeficiente de cada pais de posible destino como la peso del vector dividio por la distancia al vector ogirinal y el pais que mas coeficiente saque será el recomendado

un ejemplo sencillo de lo que se hace acontinuacion se puede ver el el notebook modelo_parte_2 que hace un caso sencillo de como se realiza lo descrito anteriormente

In [26]:
#agrupamos todos los vectores iguales

In [10]:
destinos = datos.columns[2:].tolist()
agrupados = datos.groupby(destinos)['full_name'].count().reset_index()
agrupados.head()

,AT,BE,BG,CA,CH,CN,CV,CY,DE,DZ,...,RU,SA,SE,TN,TR,UA,US,UZ,ZM,full_name
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,292
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,212
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3875
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,4720


In [11]:
#preparamos los datos para entrenar el modelo
X= agrupados[destinos].values
X

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ..., 
       [1, 1, 1, ..., 1, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 0]])

In [12]:
#entrenamos el modelo
vecinos  = NearestNeighbors(n_neighbors=9,metric='cosine',algorithm='brute')
vecinos = vecinos.fit(X)


In [14]:
#este sera el valor a predecir
#datos.iloc[8,:]
predecir = np.array([datos[destinos].values[8]])

In [15]:
#sacamos las prediciones, o vectores que mas se asemejen al original
distances, indices = vecinos.kneighbors(predecir)
datos.iloc[indices[0],:]


,full_name,off_country_code,AT,BE,BG,CA,CH,CN,CV,CY,...,PT,RU,SA,SE,TN,TR,UA,US,UZ,ZM
358,BQWWM YMFUMEEGHMZFA/VAZU,BE|NO,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
384,BRMRR/MZZQXUMZZQXU MMXFA,SE|BE,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
21462,/DQUZSMDPMZQIMZFQD,BE|DZ,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
379,BQZFFU YMFUMEYAFFAZQZ/YUWWA,ES|BE|BE|US|US|TR|BE|BE|BE,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
381,BQZFFU YMFUMEYMVMZPQD/ZUOA,IT|BE,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
382,BQZFFU ZQEFADUTQUWWUZQZ/VMDA,BE|DZ,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
383,BQZM/RDMZOUEOASAZLMXQL PQ XM,RU|RU|BE,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
385,BTAZSHA/PGK,BE|NO|NO|BE,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
410,BUDVA TMZZQXQWAUEFUZQZ/QUVM,DE|BE,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
#funcion auxiliar para restar dos listas coordenada a coordenada
def restar(a,b):
    lon = len(a)
    sol = [0]*lon
    for i in range(lon):
        sol[i] = a[i]-b[i]
    return sol

In [17]:
#funcion auxiliar para sumar dos listas coordenada a coordenada
def suma(a,b):
    lon = len(a)
    sol = [0]*lon
    for i in range(lon):
        sol[i] = a[i]+b[i]
    return sol

In [22]:
#funcion que calcula el pais con mas rating y acaba recomendandonoslo
def dar_destino(listaCandidatos,pesos,distances,predecir):
    destinos = []
    for i in listaCandidatos:
        aux = restar(i,predecir)
        destinos.append(aux) 
    destinoPesos = []
    for i in range(len(pesos)):
        distancia = distances[i]
        peso = pesos[i]
        destino = destinos[i]
        destinoPesos.append(map(lambda x: x*peso,destino))
    sol = reduce(lambda x,y:suma(x,y),destinoPesos,[0]*len(destinoPesos[0]))
    destino = sol.index(max(sol))
    return destino

In [20]:
#definimos las variables para sacar resultado
candidatos = agrupados.iloc[indices[0],:][destinos].values
pesos = agrupados.iloc[indices[0],:]['full_name'].values

In [23]:
#obtenemos el resultado
destino = dar_destino(candidatos,pesos,distances[0],predecir[0])
destinos[destino]

'GB'